In [211]:
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import numpy as np

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes #uncomment if geopy has not been installed 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes #uncomment if folium has not been installed 
import folium # map rendering library

#!pip install BeautifulSoup4
from urllib.request import urlopen
from bs4 import BeautifulSoup as bs
import ssl
import csv

import urllib.parse
import urllib.error
import ast
import os
from urllib.request import Request, urlopen

#to extract the csv file of latitude and longitude 
import io 

#to deal with regex - regular expressions
import re

print('Libraries imported.')

Libraries imported.


In [249]:
from IPython.display import HTML
import random

def hide_toggle(for_next=False):
    this_cell = """$('div.cell.code_cell.rendered.selected')"""
    next_cell = this_cell + '.next()'

    toggle_text = 'Toggle show/hide'  # text shown on toggle link
    target_cell = this_cell  # target cell to control with toggle
    js_hide_current = ''  # bit of JS to permanently hide code in current cell (only when toggling next cell)

    if for_next:
        target_cell = next_cell
        toggle_text += ' next cell'
        js_hide_current = this_cell + '.find("div.input").hide();'

    js_f_name = 'code_toggle_{}'.format(str(random.randint(1,2**64)))

    html = """
        <script>
            function {f_name}() {{
                {cell_selector}.find('div.input').toggle();
            }}

            {js_hide_current}
        </script>

        <a href="javascript:{f_name}()">{toggle_text}</a>
    """.format(
        f_name=js_f_name,
        cell_selector=target_cell,
        js_hide_current=js_hide_current, 
        toggle_text=toggle_text
    )

    return HTML(html)

In [250]:
CLIENT_ID = 'TN32UEOPQPWXMDYXAIEPZ3C4G0Q2XG1KOI3K4P5BNBV5NIVL' # your Foursquare ID
CLIENT_SECRET = 'VXCCMMIUMDRIBFPFRPFYT3VK43BHWZ5EIS5UIPVGMITSWDK1' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

hide_toggle()

Your credentails:
CLIENT_ID: TN32UEOPQPWXMDYXAIEPZ3C4G0Q2XG1KOI3K4P5BNBV5NIVL
CLIENT_SECRET:VXCCMMIUMDRIBFPFRPFYT3VK43BHWZ5EIS5UIPVGMITSWDK1


In [36]:
#downloaded data from http://zipatlas.com/us/mn/minneapolis.htm

df = pd.read_csv('minneapolis_df.csv')
df.head()  

,#,Zip Code,Location,City,Population,% In College,National Rank
0,1,55402,"44.975968, -93.271425","Minneapolis, Minnesota",176,100.00 %,#18
1,2,55455,"44.971734, -93.235244","Minneapolis, Minnesota","2,036",100.00 %,#19
2,3,55414,"44.978043, -93.223019","Minneapolis, Minnesota","24,126",90.43 %,#147
3,4,55401,"44.984334, -93.268720","Minneapolis, Minnesota","3,649",75.10 %,#281
4,5,55403,"44.970570, -93.284032","Minneapolis, Minnesota","14,873",73.67 %,#309


In [37]:
df[['latitude','longitude']] = df.Location.str.split(",",expand=True) 
df.head()

,#,Zip Code,Location,City,Population,% In College,National Rank,latitude,longitude
0,1,55402,"44.975968, -93.271425","Minneapolis, Minnesota",176,100.00 %,#18,44.975968,-93.271425
1,2,55455,"44.971734, -93.235244","Minneapolis, Minnesota","2,036",100.00 %,#19,44.971734,-93.235244
2,3,55414,"44.978043, -93.223019","Minneapolis, Minnesota","24,126",90.43 %,#147,44.978043,-93.223019
3,4,55401,"44.984334, -93.268720","Minneapolis, Minnesota","3,649",75.10 %,#281,44.984334,-93.268720
4,5,55403,"44.970570, -93.284032","Minneapolis, Minnesota","14,873",73.67 %,#309,44.970570,-93.284032


In [38]:
df.drop(['Location', 'City', '#', 'National Rank'], axis = 1, inplace = True)
df.head() 

,Zip Code,Population,% In College,latitude,longitude
0,55402,176,100.00 %,44.975968,-93.271425
1,55455,"2,036",100.00 %,44.971734,-93.235244
2,55414,"24,126",90.43 %,44.978043,-93.223019
3,55401,"3,649",75.10 %,44.984334,-93.268720
4,55403,"14,873",73.67 %,44.970570,-93.284032


In [39]:
df.rename(columns={"Zip Code": "zipcode", "Population": "population", "% In College": "percentCollege"}, inplace = True)
df.head()

,zipcode,population,percentCollege,latitude,longitude
0,55402,176,100.00 %,44.975968,-93.271425
1,55455,"2,036",100.00 %,44.971734,-93.235244
2,55414,"24,126",90.43 %,44.978043,-93.223019
3,55401,"3,649",75.10 %,44.984334,-93.268720
4,55403,"14,873",73.67 %,44.970570,-93.284032


In [40]:
exp = re.compile(r'%')
df['percentCollege'] = [exp.sub('', x) for x in df['percentCollege']]
df.head()

,zipcode,population,percentCollege,latitude,longitude
0,55402,176,100.00,44.975968,-93.271425
1,55455,"2,036",100.00,44.971734,-93.235244
2,55414,"24,126",90.43,44.978043,-93.223019
3,55401,"3,649",75.10,44.984334,-93.268720
4,55403,"14,873",73.67,44.970570,-93.284032


In [41]:
df.dtypes

zipcode            int64
population        object
percentCollege    object
latitude          object
longitude         object
dtype: object

In [42]:
#remove , in population to get it ready for type conversion from object to int

exp = re.compile(r',')
df['population'] = [exp.sub('', x) for x in df['population']]
df.head()

,zipcode,population,percentCollege,latitude,longitude
0,55402,176,100.00,44.975968,-93.271425
1,55455,2036,100.00,44.971734,-93.235244
2,55414,24126,90.43,44.978043,-93.223019
3,55401,3649,75.10,44.984334,-93.268720
4,55403,14873,73.67,44.970570,-93.284032


In [43]:
#convert population, percentCollege, latitude, and longitude to int or float
df[["population", "percentCollege", "latitude", "longitude"]] = df[["population","percentCollege", "latitude", "longitude"]].apply(pd.to_numeric)


In [44]:
df.dtypes

zipcode             int64
population          int64
percentCollege    float64
latitude          float64
longitude         float64
dtype: object

In [95]:
df.sort_values(by='zipcode')

,zipcode,population,percentCollege,latitude,longitude
3,55401,3649,75.10,44.984334,-93.268720
0,55402,176,100.00,44.975968,-93.271425
4,55403,14873,73.67,44.970570,-93.284032
6,55404,27282,46.47,44.962198,-93.262011
9,55405,15909,39.73,44.971865,-93.302502
12,55406,33033,29.39,44.938334,-93.221638
34,55407,37879,20.17,44.935036,-93.253029
8,55408,30564,43.92,44.943815,-93.293867
17,55409,21100,26.28,44.923939,-93.283436
14,55410,18435,27.40,44.913345,-93.316942


In [45]:
address = 'Minneapolis, Minnesota'

geolocator = Nominatim(user_agent="minneapolis_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Minneapolis are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Minneapolis are 44.9772995, -93.2654692.


In [46]:
# create map of Minneaplis using latitude and longitude values
map_minn_zipcode = folium.Map(location = [latitude, longitude], zoom_start = 10)

# add markers to map
for lat, lng, zipcode in zip(df['latitude'], df['longitude'], df['zipcode']):
    label = '{}'.format(zipcode)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_minn_zipcode)  
    
map_minn_zipcode

In [157]:
#scrape neighborhoods in Minneapolis from wikipedia using BeautifulSoup as bs

url = requests.get("https://en.wikipedia.org/wiki/Category:Neighborhoods_in_Minneapolis")

soup = bs(url.text, "html.parser")

neighborhood = []
for link in soup.find_all("a"):
    neighbor = link.get("title")
    neighborhood.append(neighbor)

neighborhood = list(filter(None.__ne__, neighborhood)) #remove Nonetype in list of neighborhoods
neighborhood = pd.DataFrame(neighborhood)

#cleanup the neighborhood dataframe 
neighborhoodDict = {"neighborhood": []}
for row, neighbor in neighborhood.iterrows(): 
    if 'Minneapolis' in neighbor[0]:
        neighborhoodDict["neighborhood"].append(neighbor[0])

neighborhoodMinn = pd.DataFrame(neighborhoodDict)

#remove rows that are not neighborhoods
neighborhoodMinn = neighborhoodMinn.drop(neighborhoodMinn.index[[0, 1, 2, 3, 91, 92]]).reset_index(drop=True)

#cleanup some names of neighborhood in order to locate it later for latitude and longitude

exp = re.compile(r' \(neighborhood\)')
neighborhoodMinn['neighborhood'] = [exp.sub('', x) for x in neighborhoodMinn['neighborhood']]

#Mill District results in Nonetype latitude and longitude values so drop its row
neighborhoodMinn = neighborhoodMinn.drop(neighborhoodMinn.index[[51]]).reset_index(drop=True)

#remove Ventura Village because it's located outside Minneapolis - according to FourSquare API
neighborhoodMinn = neighborhoodMinn.drop(neighborhoodMinn.index[[76]]).reset_index(drop=True)

#Nicollet Island- East Bank, Minneapolis' returns a NoneType error, so therefore changed it to Nicollet Island
neighborhoodMinn['neighborhood'] = neighborhoodMinn['neighborhood'].replace(['Nicollet Island/East Bank, Minneapolis'],'Nicollet Island, Minneapolis')

neighborhoodMinn

,neighborhood
0,"Armatage, Minneapolis"
1,"Audubon Park, Minneapolis"
2,"Bancroft, Minneapolis"
3,"Beltrami, Minneapolis"
4,"Bottineau, Minneapolis"
5,"Bryant, Minneapolis"
6,"Bryn Mawr, Minneapolis"
7,"Cedar-Isles-Dean, Minneapolis"
8,"Cedar-Riverside, Minneapolis"
9,"Central, Minneapolis"


In [161]:
#total number of neighborhoods = 85
neighborhoodMinn.shape

(85, 1)

In [176]:
# NOTE TO MYSELF: DO NOT RUN THIS CODE CHUNK

#this code block takes ~ 45 seconds on my machine due to having to locate latitude and longitude for 86 neighborhoods

latitude = []
longitude = []

geolocator = Nominatim(user_agent="minneapolis_explorer")
for row, neighbor in neighborhoodMinn.iterrows(): 
    location = geolocator.geocode(neighbor[0])
    latitude.append(location.latitude)
    longitude.append(location.longitude)

neighborhoodMinn['latitude'] = latitude
neighborhoodMinn['longitude'] = longitude
neighborhoodMinn.head()

,neighborhood,latitude,longitude
0,"Armatage, Minneapolis",44.897236,-93.310342
1,"Audubon Park, Minneapolis",45.018883,-93.236355
2,"Bancroft, Minneapolis",44.930570,-93.254929
3,"Beltrami, Minneapolis",44.995352,-93.242513
4,"Bottineau, Minneapolis",45.010566,-93.269304


In [177]:
neighborhoodMinn.dtypes

neighborhood     object
latitude        float64
longitude       float64
dtype: object

In [178]:
address = 'Minneapolis, Minnesota'
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Minneapolis are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Minneapolis are 44.9772995, -93.2654692.


In [179]:
# create map of Minneaplis using latitude and longitude values by neighborhood
map_minn_neighborhood = folium.Map(location = [latitude, longitude], zoom_start = 12)

# add markers to map
for lat, lng, neighborhood in zip(neighborhoodMinn['latitude'], neighborhoodMinn['longitude'], neighborhoodMinn['neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_minn_neighborhood)  
    
map_minn_neighborhood

In [180]:
neighborhoodMinn.loc[0, 'latitude']

44.8972364

In [168]:
neighborhoodMinn.loc[0, 'latitude']

neighborhood    Armatage, Minneapolis
Name: 0, dtype: object

In [181]:
neighborhood_latitude = neighborhoodMinn.loc[0, 'latitude'] # neighborhood latitude value
neighborhood_longitude = neighborhoodMinn.loc[0, 'longitude'] # neighborhood longitude value

neighborhood_name = neighborhoodMinn.loc[0, 'neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Armatage, Minneapolis are 44.8972364, -93.3103424.


In [189]:
#get the top 100 venues that are in Armatage, Minneapolis within a radius of 500 meters.

LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
#url # display URL


'https://api.foursquare.com/v2/venues/explore?&client_id=TN32UEOPQPWXMDYXAIEPZ3C4G0Q2XG1KOI3K4P5BNBV5NIVL&client_secret=VXCCMMIUMDRIBFPFRPFYT3VK43BHWZ5EIS5UIPVGMITSWDK1&v=20180604&ll=44.8972364,-93.3103424&radius=500&limit=100'

In [190]:
results = requests.get(url).json()
#results

In [191]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [192]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Armatage Park,Park,44.899459,-93.310883
1,Domino's Pizza,Pizza Place,44.894770,-93.308208
2,"Wagner Greenhouses, Inc.",Flower Shop,44.892827,-93.309378
3,Armatage Skate Park,Skate Park,44.899742,-93.310570
4,Super USA,Convenience Store,44.894608,-93.308577


In [193]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

5 venues were returned by Foursquare.


In [194]:
#function to repeat the same process to all the neighborhoods in Minneapolis

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [216]:
minneapolis_venues = getNearbyVenues(names=neighborhoodMinn['neighborhood'],
                                   latitudes=neighborhoodMinn['latitude'],
                                   longitudes=neighborhoodMinn['longitude']
                                  )

print(minneapolis_venues.shape)

Armatage, Minneapolis
Audubon Park, Minneapolis
Bancroft, Minneapolis
Beltrami, Minneapolis
Bottineau, Minneapolis
Bryant, Minneapolis
Bryn Mawr, Minneapolis
Cedar-Isles-Dean, Minneapolis
Cedar-Riverside, Minneapolis
Central, Minneapolis
Cleveland, Minneapolis
Columbia Park, Minneapolis
Como, Minneapolis
Cooper, Minneapolis
Corcoran, Minneapolis
Diamond Lake, Minneapolis
Downtown East, Minneapolis
Downtown West, Minneapolis
East Calhoun, Minneapolis
East Harriet, Minneapolis
East Isles, Minneapolis
East Phillips, Minneapolis
Elliot Park, Minneapolis
Ericsson, Minneapolis
Field, Minneapolis
Folwell, Minneapolis
Fulton, Minneapolis
Hale, Minneapolis
Harrison, Minneapolis
Hawthorne, Minneapolis
Hiawatha, Minneapolis
Holland, Minneapolis
Howe, Minneapolis
Jordan, Minneapolis
Keewaydin, Minneapolis
Kenny, Minneapolis
Kenwood, Minneapolis
King Field, Minneapolis
Lind-Bohanon, Minneapolis
Linden Hills, Minneapolis
Logan Park, Minneapolis
Longfellow, Minneapolis
Lowry Hill East, Minneapolis
Lo

In [217]:
minneapolis_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Armatage, Minneapolis",44.897236,-93.310342,Armatage Park,44.899459,-93.310883,Park
1,"Armatage, Minneapolis",44.897236,-93.310342,Domino's Pizza,44.894770,-93.308208,Pizza Place
2,"Armatage, Minneapolis",44.897236,-93.310342,"Wagner Greenhouses, Inc.",44.892827,-93.309378,Flower Shop
3,"Armatage, Minneapolis",44.897236,-93.310342,Armatage Skate Park,44.899742,-93.310570,Skate Park
4,"Armatage, Minneapolis",44.897236,-93.310342,Super USA,44.894608,-93.308577,Convenience Store


In [218]:
#how many venues were returned for each neighborhood

minneapolis_venues.groupby('Neighborhood').count()

#problem: some neighborhoods are very small. eg: Hawthorne, Minneapolis	only has 2 venues 
#to think about: do we want to drop neighborhoods that are very small, or increase the radius?

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Armatage, Minneapolis",5,5,5,5,5,5
"Audubon Park, Minneapolis",18,18,18,18,18,18
"Bancroft, Minneapolis",5,5,5,5,5,5
"Beltrami, Minneapolis",10,10,10,10,10,10
"Bottineau, Minneapolis",20,20,20,20,20,20
"Bryant, Minneapolis",7,7,7,7,7,7
"Bryn Mawr, Minneapolis",7,7,7,7,7,7
"Cedar-Isles-Dean, Minneapolis",8,8,8,8,8,8
"Cedar-Riverside, Minneapolis",29,29,29,29,29,29


In [203]:
#how many unique categories can be curated from all the returned values

print('There are {} uniques categories.'.format(len(minneapolis_venues['Venue Category'].unique())))

There are 244 uniques categories.


In [204]:
#one hot encoding to put the unique categories in columns
minneapolis_onehot = pd.get_dummies(minneapolis_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
minneapolis_onehot['Neighborhood'] = minneapolis_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [minneapolis_onehot.columns[-1]] + list(minneapolis_onehot.columns[:-1])
minneapolis_onehot = minneapolis_onehot[fixed_columns]

minneapolis_onehot.head()

,Neighborhood,ATM,Acupuncturist,Advertising Agency,African Restaurant,American Restaurant,Antique Shop,Arepa Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Bath House,Beach,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bistro,Board Shop,Bookstore,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Building,Burger Joint,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Caribbean Restaurant,Carpet Store,Casino,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Rec Center,College Residence Hall,Comedy Club,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Diner,Discount Store,Distillery,Dive Bar,Dog Run,Donut Shop,Eastern European Restaurant,Electronics Store,Elementary School,English Restaurant,Event Space,Fabric Shop,Farm,Farmers Market,Fast Food Restaurant,Film Studio,Fish & Chips Shop,Fishing Spot,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Football Stadium,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hardware Store,Health & Beauty Service,High School,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hospital,Hostel,Hot Dog Joint,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karaoke Bar,Kebab Restaurant,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Library,Light Rail Station,Liquor Store,Malay Restaurant,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Motorcycle Shop,Movie Theater,Moving Target,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,Non-Profit,Noodle House,Office,Opera House,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Outdoor Supply Store,Outdoors & Recreation,Paper / Office Supplies Store,Park,Performing Arts Venue,Pet Café,Pet Store,Pharmacy,Piano Bar,Pilates Studio,Pizza Place,Playground,Plaza,Pool,Print Shop,Pub,Public Art,Ramen Restaurant,Record Shop,Restaurant,Road,Rock Club,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Store,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,Southern / Soul Food Restaurant,Spa,Speakeasy,Spiritual Center,Sporting Goods Shop,Sports Bar,Sports Club,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Szechuan Restaurant,Taco Place,Tailor Shop,Tanning Salon,Tattoo Parlor,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Theme Restaurant,Thrift / Vintage Store,Tour Provider,Tourist Information Center,Toy / Game Store,Trail,Train,Travel & Transport,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Vineyard,Volleyball Court,Warehouse Store,Wine Bar,Winery,Wings Joint,Women's Store,Yoga Studio
0,"Armatage, Minneapolis",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Armatage, Minneapolis",0,0,0,0,0,0,0,0,0,0

In [205]:
minneapolis_onehot.shape

(1284, 245)

In [206]:
#group rows by neighborhood, take the mean of the frequency of occurrence of each category
 
minneapolis_grouped = minneapolis_onehot.groupby('Neighborhood').mean().reset_index()
minneapolis_grouped

,Neighborhood,ATM,Acupuncturist,Advertising Agency,African Restaurant,American Restaurant,Antique Shop,Arepa Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Bath House,Beach,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bistro,Board Shop,Bookstore,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Building,Burger Joint,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Caribbean Restaurant,Carpet Store,Casino,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Rec Center,College Residence Hall,Comedy Club,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Diner,Discount Store,Distillery,Dive Bar,Dog Run,Donut Shop,Eastern European Restaurant,Electronics Store,Elementary School,English Restaurant,Event Space,Fabric Shop,Farm,Farmers Market,Fast Food Restaurant,Film Studio,Fish & Chips Shop,Fishing Spot,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Football Stadium,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hardware Store,Health & Beauty Service,High School,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hospital,Hostel,Hot Dog Joint,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karaoke Bar,Kebab Restaurant,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Library,Light Rail Station,Liquor Store,Malay Restaurant,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Motorcycle Shop,Movie Theater,Moving Target,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,Non-Profit,Noodle House,Office,Opera House,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Outdoor Supply Store,Outdoors & Recreation,Paper / Office Supplies Store,Park,Performing Arts Venue,Pet Café,Pet Store,Pharmacy,Piano Bar,Pilates Studio,Pizza Place,Playground,Plaza,Pool,Print Shop,Pub,Public Art,Ramen Restaurant,Record Shop,Restaurant,Road,Rock Club,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Store,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,Southern / Soul Food Restaurant,Spa,Speakeasy,Spiritual Center,Sporting Goods Shop,Sports Bar,Sports Club,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Szechuan Restaurant,Taco Place,Tailor Shop,Tanning Salon,Tattoo Parlor,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Theme Restaurant,Thrift / Vintage Store,Tour Provider,Tourist Information Center,Toy / Game Store,Trail,Train,Travel & Transport,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Vineyard,Volleyball Court,Warehouse Store,Wine Bar,Winery,Wings Joint,Women's Store,Yoga Studio
0,"Armatage, Minneapolis",0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.00,0.000000,0.000000,0.000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.200000,0.000000,0.000,0.000000,0.000000,0.00000

In [219]:
minneapolis_grouped.shape

(85, 245)

In [220]:
#print each neighborhood along with the top 5 most common venues

num_top_venues = 10

for hood in minneapolis_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = minneapolis_grouped[minneapolis_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

ndscaping  0.22
1                Intersection  0.11
2           Food & Drink Shop  0.11
3              Discount Store  0.11
4         American Restaurant  0.11
5              Sandwich Place  0.11
6                        Park  0.11
7                   Gift Shop  0.11
8       Outdoors & Recreation  0.00
9        Outdoor Supply Store  0.00


----Keewaydin, Minneapolis----
                     venue  freq
0              Yoga Studio  0.17
1              Coffee Shop  0.17
2                 Boutique  0.17
3                Gift Shop  0.17
4                    Beach  0.17
5                     Park  0.17
6                    Plaza  0.00
7     Outdoor Supply Store  0.00
8  New American Restaurant  0.00
9                Nightclub  0.00


----Kenny, Minneapolis----
                        venue  freq
0                     Brewery  0.25
1                        Park  0.25
2  Construction & Landscaping  0.25
3        Gym / Fitness Center  0.25
4     New American Restaurant  0.00
5                  

In [221]:
#function to sort the venues in descending order

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [236]:
#create the new dataframe and display the top 10 venues for each neighborhood

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = minneapolis_grouped['Neighborhood']

for ind in np.arange(minneapolis_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(minneapolis_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Armatage, Minneapolis",Skate Park,Convenience Store,Park,Pizza Place,Flower Shop,Film Studio,Fabric Shop,Farm,Farmers Market,Fast Food Restaurant
1,"Audubon Park, Minneapolis",Pharmacy,Cosmetics Shop,Convenience Store,Hotel Bar,Jewelry Store,Park,Coffee Shop,Clothing Store,Chinese Restaurant,Movie Theater
2,"Bancroft, Minneapolis",Caribbean Restaurant,Ice Cream Shop,Discount Store,Garden Center,Southern / Soul Food Restaurant,Yoga Studio,Fishing Spot,Fast Food Restaurant,Film Studio,Fish & Chips Shop
3,"Beltrami, Minneapolis",Brewery,Chinese Restaurant,Coffee Shop,Gift Shop,Gay Bar,Asian Restaurant,Café,Playground,Food Truck,Liquor Store
4,"Bottineau, Minneapolis",Park,Theme Restaurant,Bar,Electronics Store,Coffee Shop,German Restaurant,Liquor Store,Gym,BBQ Joint,Beer Garden


In [237]:
#cluster neighborhoods

# set number of clusters
kclusters = 5

minneapolis_grouped_clustering = minneapolis_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(minneapolis_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [238]:
#create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood

# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

minneapolis_merged = neighborhoodMinn

# merge minneapolis_grouped with neighborhoodMinn to add latitude/longitude for each neighborhood
minneapolis_merged = minneapolis_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='neighborhood')

minneapolis_merged.head() # check the last columns!

#problem with this is that some of the neighborhoods have the later common venue with frequency 0, meaning these venues are
#there to fill up spaces and do not actually exist within the radius of 500

,neighborhood,latitude,longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Armatage, Minneapolis",44.897236,-93.310342,3,Skate Park,Convenience Store,Park,Pizza Place,Flower Shop,Film Studio,Fabric Shop,Farm,Farmers Market,Fast Food Restaurant
1,"Audubon Park, Minneapolis",45.018883,-93.236355,0,Pharmacy,Cosmetics Shop,Convenience Store,Hotel Bar,Jewelry Store,Park,Coffee Shop,Clothing Store,Chinese Restaurant,Movie Theater
2,"Bancroft, Minneapolis",44.930570,-93.254929,0,Caribbean Restaurant,Ice Cream Shop,Discount Store,Garden Center,Southern / Soul Food Restaurant,Yoga Studio,Fishing Spot,Fast Food Restaurant,Film Studio,Fish & Chips Shop
3,"Beltrami, Minneapolis",44.995352,-93.242513,0,Brewery,Chinese Restaurant,Coffee Shop,Gift Shop,Gay Bar,Asian Restaurant,Café,Playground,Food Truck,Liquor Store
4,"Bottineau, Minneapolis",45.010566,-93.269304,0,Park,Theme Restaurant,Bar,Electronics Store,Coffee Shop,German Restaurant,Liquor Store,Gym,BBQ Joint,Beer Garden


In [240]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(minneapolis_merged['latitude'], minneapolis_merged['longitude'], minneapolis_merged['neighborhood'], minneapolis_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [244]:
#examine clusters

#CLUSTER 1

minneapolis_merged.loc[minneapolis_merged['Cluster Labels'] == 0, minneapolis_merged.columns[[0] + list(range(4, minneapolis_merged.shape[1]))]]

,neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,"Audubon Park, Minneapolis",Pharmacy,Cosmetics Shop,Convenience Store,Hotel Bar,Jewelry Store,Park,Coffee Shop,Clothing Store,Chinese Restaurant,Movie Theater
2,"Bancroft, Minneapolis",Caribbean Restaurant,Ice Cream Shop,Discount Store,Garden Center,Southern / Soul Food Restaurant,Yoga Studio,Fishing Spot,Fast Food Restaurant,Film Studio,Fish & Chips Shop
3,"Beltrami, Minneapolis",Brewery,Chinese Restaurant,Coffee Shop,Gift Shop,Gay Bar,Asian Restaurant,Café,Playground,Food Truck,Liquor Store
4,"Bottineau, Minneapolis",Park,Theme Restaurant,Bar,Electronics Store,Coffee Shop,German Restaurant,Liquor Store,Gym,BBQ Joint,Beer Garden
5,"Bryant, Minneapolis",Deli / Bodega,Moving Target,Food Truck,American Restaurant,Taco Place,Park,Fishing Spot,Fast Food Restaurant,Film Studio,Fish & Chips Shop
6,"Bryn Mawr, Minneapolis",Grocery Store,Furniture / Home Store,Pizza Place,Coffee Shop,Intersection,Men's Store,Sandwich Place,Film Studio,Farm,Farmers Market
7,"Cedar-Isles-Dean, Minneapolis",Sporting Goods Shop,Bridge,Speakeasy,American Restaurant,Beach,Tourist Information Center,Food,Gym / Fitness Center,Yoga Studio,Film Studio
8,"Cedar-Riverside, Minneapolis",Theater,Sandwich Place,Bar,Chinese Restaurant,Greek Restaurant,Thai Restaurant,Light Rail Station,Bus Station,Music Venue,ATM
9,"Central, Minneapolis",Coffee Shop,Hotel,Food Truck,Salad Place,Bar,Burger Joint,Office,Japanese Restaurant,New American Restaurant,Music School
10,"Cleveland, Minneapolis",Food Truck,Music Venue,Trail,Carpet Store,Yoga Studio,Fabric Shop,Food Court,Food & Drink Shop,Food,Flower Shop


In [245]:
#CLUSTER 2

minneapolis_merged.loc[minneapolis_merged['Cluster Labels'] == 1, minneapolis_merged.columns[[0] + list(range(4, minneapolis_merged.shape[1]))]]

,neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
79,"Wenonah, Minneapolis",Road,Baseball Field,Yoga Studio,Event Space,Food Truck,Food Court,Food & Drink Shop,Food,Flower Shop,Fishing Spot


In [246]:
#CLUSTER 3

minneapolis_merged.loc[minneapolis_merged['Cluster Labels'] == 2, minneapolis_merged.columns[[0] + list(range(4, minneapolis_merged.shape[1]))]]

,neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,"East Harriet, Minneapolis",Park,Garden,Pub,American Restaurant,Carpet Store,Other Great Outdoors,Food,Flower Shop,Fabric Shop,Fish & Chips Shop
60,"Powderhorn Park, Minneapolis",Bakery,Arts & Crafts Store,Caribbean Restaurant,Park,Video Store,Garden,Food & Drink Shop,Food,Flower Shop,Fishing Spot
78,"Webber-Camden, Minneapolis",Bakery,Convenience Store,Park,Event Space,Food Truck,Food Court,Food & Drink Shop,Food,Flower Shop,Fishing Spot


In [247]:
#CLUSTER 4

minneapolis_merged.loc[minneapolis_merged['Cluster Labels'] == 3, minneapolis_merged.columns[[0] + list(range(4, minneapolis_merged.shape[1]))]]

,neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Armatage, Minneapolis",Skate Park,Convenience Store,Park,Pizza Place,Flower Shop,Film Studio,Fabric Shop,Farm,Farmers Market,Fast Food Restaurant
14,"Corcoran, Minneapolis",Outdoor Supply Store,Theater,Bakery,American Restaurant,Park,Train,Film Studio,Farm,Farmers Market,Fast Food Restaurant
15,"Diamond Lake, Minneapolis",Breakfast Spot,Acupuncturist,Park,Grocery Store,Business Service,Fishing Spot,Farmers Market,Fast Food Restaurant,Film Studio,Fish & Chips Shop
23,"Ericsson, Minneapolis",Park,Home Service,Lake,Playground,Film Studio,Fabric Shop,Farm,Farmers Market,Fast Food Restaurant,Fishing Spot
28,"Harrison, Minneapolis",American Restaurant,Grocery Store,Convenience Store,Park,Vegetarian / Vegan Restaurant,Fish & Chips Shop,Fabric Shop,Farm,Farmers Market,Fast Food Restaurant
35,"Kenny, Minneapolis",Park,Brewery,Gym / Fitness Center,Construction & Landscaping,Fish & Chips Shop,Farm,Farmers Market,Fast Food Restaurant,Film Studio,Fishing Spot
37,"King Field, Minneapolis",Park,Yoga Studio,Salon / Barbershop,Thrift / Vintage Store,American Restaurant,Toy / Game Store,Coffee Shop,Farm,Farmers Market,Fast Food Restaurant
38,"Lind-Bohanon, Minneapolis",Park,Mexican Restaurant,Ice Cream Shop,Home Service,BBQ Joint,Farm,Farmers Market,Fast Food Restaurant,Film Studio,Fish & Chips Shop
39,"Linden Hills, Minneapolis",Park,Breakfast Spot,Comfort Food Restaurant,Thai Restaurant,Bakery,Tennis Court,Coffee Shop,Snack Place,Flower Shop,Spa
47,"Marshall Terrace, Minneapolis",Brewery,Miscellaneous Shop,Music Venue,Paper / Office Supplies Store,Park,Yoga Studio,Fish & Chips Shop,Fast Food Restaurant,Film Studio,Flower Shop


In [248]:
#CLUSTER 5

minneapolis_merged.loc[minneapolis_merged['Cluster Labels'] == 4, minneapolis_merged.columns[[0] + list(range(4, minneapolis_merged.shape[1]))]]

,neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
29,"Hawthorne, Minneapolis",Construction & Landscaping,Vineyard,Yoga Studio,Event Space,Food Truck,Food Court,Food & Drink Shop,Food,Flower Shop,Fishing Spot


In [ ]:
# work on the best number of clusters instead of assuming 5